In [1]:
from MAESeq_utils import dataloader,get_dict, seq_data_to_onehot

seq_list, max_len = dataloader(len_data=1000, max_len_percintile=75)
dict_char2int, dict_int2char = get_dict(seq_list)
onehot_data = seq_data_to_onehot(seq_list,dict_char2int,max_len)
dimension = len(dict_char2int)

print("Length of seq is {}".format(max_len))
print("Dimension is {}".format(dimension))


In [15]:
from MARSeq_model import AutoencoderGRU, my_loss_entro, calReconstructRate, evaluateReconstructRate_varied_length
import keras
import tensorflow as tf

# autoencoder = keras.models.load_model('trained_model/',custom_objects={
#     'my_loss_entro':my_loss_entro, 
#     'calReconstructRate':calReconstructRate})
evaluateReconstructRate_varied_length(onehot_data[0:100], onehot_data[0:100])
